<a href="https://colab.research.google.com/github/Edith1100/dw_matrix_road_sign/blob/master/D5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [15]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [4]:
cd 'drive/My Drive/Colab Notebooks/matrix/matrix three/dw_matrix_road_sign/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose = params_fit.get('verbose',1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(0.3),
  
  Dense(1024, activation='relu') ,
  Dropout(0.3),

  Dense(num_classes, activation='softmax') 

  ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.5949 - accuracy: 0.2910 - val_loss: 1.0773 - val_accuracy: 0.6707
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.8306 - accuracy: 0.7302 - val_loss: 0.1996 - val_accuracy: 0.9374
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3566 - accuracy: 0.8897 - val_loss: 0.1128 - val_accuracy: 0.9713
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2171 - accuracy: 0.9340 - val_loss: 0.0480 - val_accuracy: 0.9854
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1668 - accuracy: 0.9511 - val_loss: 0.0302 - val_accuracy: 0.9915


0.973015873015873

In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.0903 - accuracy: 0.9730


[0.09027792513370514, 0.9730158448219299]

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),
  Dense(1024, activation='relu') ,
  Dropout(params['dropout_dense_block_one']),
  
  Dense(1024, activation='relu') ,
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax') 

  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = 128,#int(params.get('batch_size', 128)),
      epochs=5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy,
          'status' : STATUS_OK,
          'model': model}

In [36]:
space ={
    'batch_size': hp.quniform('batch_size', 100, 200,10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.42467861142516283, 'dropout_cnn_block_three': 0.46994952570830395, 'dropout_cnn_block_two': 0.3135399089512582, 'dropout_dense_block_one': 0.32451860928685156, 'dropout_dense_block_two': 0.38252363233393827}
accuracy=0.978458046913147
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.33540957380464304, 'dropout_cnn_block_three': 0.33054909530125653, 'dropout_cnn_block_two': 0.36487002903179616, 'dropout_dense_block_one': 0.6884470699483072, 'dropout_dense_block_two': 0.48350183161779914}
accuracy=0.9628117680549622
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4055192198191556, 'dropout_cnn_block_three': 0.43284252366723075, 'dropout_cnn_block_two': 0.37601388944946584, 'dropout_dense_block_one': 0.4408758843713416, 'dropout_dense_block_two': 0.49823884610568414}
accuracy=0.9684807062149048
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3683958193546462, 'dropout_cnn_block_three': 0.45816206618168853, 'dropout_cnn_block_two': 0.44699